In [10]:
# Dependencies
using StatsBase
using LaTeXStrings
using JLD
using GLMNet
using Plots 

In [11]:
# include Machine learning definitions and functions
include("../src/MultiplyScattering_learn_moments.jl");

In [12]:
# Load training data
dataname=  "bunnytime8"; # bunnytime-low
train = load_moments(string(dataname,".mnts")); #bunnytime8 

loading:/home/rgower/Dropbox/Scattering-1.5D/numerics/mom_linear_regression/test/../../Data/bunnytime8.mnts.jld


In [13]:
# Parameters for cross-validation
momnums = [1,2];   # Choose the moments to be used during training
options = OptionsML(momnums,[],false); 
lambdasv =  [ 2.0^-24.0, 2.0^-16.0, 2.0^-8.0,  2.0^-4.0, 2.0^-2.0, 2.0^-1.0, 1.0,  2.0^1.0, 2.0^2.0, 2.0^4.0 , 2.0^6.0 ]; # the grid of possible lambdas
crossnum =7; #the number of parts the training data is divided into to perform the cross validation 
kerneles = ["rationquad"] #,"OrnUhlen", "gauss", 
fms = [  fmmoments ]# fmmean  ,fmofm_full, fmmoments
 kernelparams = [ 2.0^-24.0, 2.0^-16.0, 2.0^(-8.0), 2.0^(-4.0), 2.0^(-2.0), 2.0^(-1.0), 1.0,  2.0^(1.0), 2.0^2.0, 2.0^4.0, 2.0^6.0 ];

In [ ]:
# crossvalid: calculates and saves the regularization parameter lambda for each feature map and each kernel
options = OptionsML(momnums,[],false); 
for kernel in kerneles
    println("Kernel: ", kernel)
    for fm in fms
        println("Feature map: ", fm.name)
        cross_valid_inner(train, fm, options, kernel, kernelparams,crossnum,lambdasv, "radius", true, dataname)
        cross_valid_inner(train, fm, options, kernel, kernelparams,crossnum,lambdasv, "concen",true, dataname)
        cross_valid_inner(train, fm, options, kernel, kernelparams,crossnum,lambdasv, "volumefraction",true, dataname)
    end
end

Kernel: rationquad
Feature map: momentsmap
best lambda score = 0.891 was with lambda = 5.960464477539063e-8 and klparameter = 0.00390625
saving the best lambda and kernalparameter in ../data/momentsmaprationquad-radius-bunnytime8-crossvalid.jld


In [ ]:
# Load training data
datatestname = "bunnytimetest8";
test = load_moments(string(datatestname,".mnts")); 

In [ ]:
## retrieve and test cross-validated lambdas
pgfplots()
options = OptionsML(momnums,[],true); 
for kernelname in kerneles
    for fm in fms
        lambda, kernelparam = get_crossvalid_parameters(fm,kernelname,"radius", options, dataname);
        mlr = setup_ml(train, fm,  kernelname,options, "radius");
        fit_L2(mlr, options,  kernelname, kernelparam, lambda);
        Rsqdr = test_linear(test,datatestname,fm,mlr,options, "radius");
        
        lambda, kernelparam = get_crossvalid_parameters(fm,kernelname,"concen",options, dataname);
        mlv = setup_ml(train, fm,  kernelname,options, "concen");
        fit_L2(mlv, options,  kernelname, kernelparam, lambda);
        Rsqdr = test_linear(test,datatestname,fm,mlv,options, "concen");   
        
        lambda, kernelparam = get_crossvalid_parameters(fm,kernelname,"volumefraction",options, dataname);
        mlv = setup_ml(train, fm,  kernelname,options, "volumefraction");
        fit_L2(mlv, options,  kernelname, kernelparam, lambda);
        Rsqdr = test_linear(test,datatestname,fm,mlv,options, "volumefraction");         
    end
end